In [1]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import xml.etree.ElementTree as ET
from lxml import etree
import subprocess


In [2]:
def load_expression_data(expression_file, samples, genes_of_interest, temp_file="filtered_expression_data.txt"):
    """
    Load expression data and filter for relevant samples and genes.

    Parameters:
    - expression_file (str): Path to the expression data file.
    - samples (list): List of sample IDs to include.
    - genes_of_interest (list): List of gene names to include.
    - temp_file (str): Temporary file to store filtered expression data.

    Returns:
    - DataFrame: Expression data filtered for the given samples and genes.
    """
    print("Filtering expression data...")

    # Read the header to find matching sample columns
    with open(expression_file, "r") as f:
        header = f.readline().strip().split('\t')

    # Determine which columns to keep
    columns_to_keep = [0] + [i for i, col in enumerate(header) if col in samples]

    if len(columns_to_keep) <= 1:
        print("No matching samples found in the header.")
        raise ValueError("No matching samples found in expression data.")

    # Write filtered data to a temporary file
    with open(temp_file, "w") as out_f:
        # Write the header
        out_f.write('\t'.join([header[i] for i in columns_to_keep]) + '\n')

        # Filter rows based on genes of interest
        for line in open(expression_file, "r"):
            split_line = line.strip().split('\t')
            gene_name = split_line[0]  # First column contains the gene names
            if gene_name in genes_of_interest:
                out_f.write('\t'.join([split_line[i] for i in columns_to_keep]) + '\n')

    # Load the filtered data into a DataFrame
    print("Loading filtered data into DataFrame...")
    expr_df = pd.read_csv(temp_file, sep='\t',header=0, encoding='latin1')

    # Return the filtered DataFrame
    return expr_df



In [7]:
def get_gdc_file_ids(project_id="TCGA-UCEC", data_type="Clinical Supplement"):
    """
    Retrieve GDC file IDs for a given project and data type.

    Parameters:
    - project_id (str): GDC project ID (e.g., "TCGA-UCEC").
    - data_type (str): Type of data to filter for (e.g., "Clinical Supplement").

    Returns:
    - list: List of file IDs.
    """
    GDC_FILES_URL = "https://api.gdc.cancer.gov/files"
    params = {
        "filters": {
            "op": "and",
            "content": [
                {"op": "in", "content": {"field": "cases.project.project_id", "value": [project_id]}},
                {"op": "in", "content": {"field": "data_category", "value": ["Clinical"]}},
                {"op": "in", "content": {"field": "data_type", "value": [data_type]}}
            ]
        },
        "format": "JSON",
        "size": 100
    }

    response = requests.post(GDC_FILES_URL, json=params)
    if response.status_code == 200:
        file_data = response.json()["data"]["hits"]
        file_ids = [file["file_id"] for file in file_data]
        print(f"Retrieved {len(file_ids)} file IDs.")
        return file_ids
    else:
        print(f"Failed to retrieve file IDs. Status code: {response.status_code}")
        return []

In [8]:
project_id = "GTEX"  # Replace with your project ID
file_ids = get_gdc_file_ids(project_id=project_id, data_type="Clinical Supplement")

# Step 2: Download XML files
if file_ids:
    download_xml_files(file_ids, xml_dir)
else:
    print("No file IDs found. Exiting.")
    exit()

Error parsing JSON: string indices must be integers, not 'str'
Response text: {"data": {"hits": [], "pagination": {"count": 0, "total": 0, "size": 10000, "from": 0, "sort": "", "page": 0, "pages": 0}}, "warnings": {}}
Error parsing JSON: string indices must be integers, not 'str'
Response text: {"data": {"hits": [], "pagination": {"count": 0, "total": 0, "size": 10000, "from": 0, "sort": "", "page": 0, "pages": 0}}, "warnings": {}}
No file IDs found for GTEx samples.


In [4]:


# Namespace mapping for XML parsing
NAMESPACES = {
    "clin_shared": "http://tcga.nci/bcr/xml/clinical/shared/2.7",
    "shared": "http://tcga.nci/bcr/xml/shared/2.7",
}

# GDC API base URL
GDC_API_URL = "https://api.gdc.cancer.gov/data/"

def download_xml_files(file_ids, output_dir):
    """
    Download XML files from the GDC Data Portal using file IDs.

    Parameters:
    - file_ids (list): List of GDC file IDs to download.
    - output_dir (str): Directory to save the downloaded XML files.

    Returns:
    - None
    """
    os.makedirs(output_dir, exist_ok=True)
    for file_id in file_ids:
        print(f"Downloading file ID: {file_id}")
        response = requests.get(GDC_API_URL + file_id, stream=True)
        if response.status_code == 200:
            content_disposition = response.headers.get("Content-Disposition", "")
            file_name = content_disposition.split("filename=")[-1].strip('"') if "filename=" in content_disposition else f"{file_id}.xml"
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Failed to download file ID: {file_id}. Status code: {response.status_code}")

def extract_race_from_xml(xml_file):
    """
    Extract race information from a single XML file using lxml.

    Parameters:
    - xml_file (str): Path to the XML file.

    Returns:
    - dict: Mapping of patient ID to race.
    """
    tree = etree.parse(xml_file)
    root = tree.getroot()

    race_data = {}
    for race_elem in root.findall(".//clin_shared:race_list/clin_shared:race", NAMESPACES):
        race = race_elem.text
        parent = race_elem.getparent().getparent()  # Move up to the patient level
        patient_id_elem = parent.find("shared:bcr_patient_barcode", NAMESPACES)
        if patient_id_elem is not None:
            patient_id = patient_id_elem.text
            # Normalize patient ID format to match phenotype file
            race_data[f"{patient_id}-01"] = race  # Add '-01' as a common suffix
    return race_data

def load_race_data(xml_dir):
    """
    Load race data from multiple XML files.

    Parameters:
    - xml_dir (str): Directory containing XML files.

    Returns:
    - DataFrame: DataFrame with patient IDs and race.
    """
    race_dict = {}
    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith(".xml"):
            full_path = os.path.join(xml_dir, xml_file)
            race_dict.update(extract_race_from_xml(full_path))
    
    return pd.DataFrame(list(race_dict.items()), columns=["sample", "race"])

def load_phenotype_data(phenotype_file, race_df):
    """
    Load phenotype data and merge with race data.

    Parameters:
    - phenotype_file (str): Path to the phenotype data file.
    - race_df (DataFrame): Race data.

    Returns:
    - DataFrame: Merged phenotype and race data.
    """
    print("Reading phenotype data...")
    phenotype_df = pd.read_csv(phenotype_file, sep='\t', encoding='latin1')

    # Normalize phenotype sample IDs to patient-level IDs
    phenotype_df['patient_id'] = phenotype_df['sample'].str.extract(r'(^TCGA-\w\w-\w\w\w\w)')

    # Merge using patient-level IDs
    merged_df = pd.merge(
        phenotype_df,
        race_df.rename(columns={'sample': 'patient_id'}),
        on='patient_id',
        how='left'
    )
    print("Merged phenotype data with race.")
    return merged_df


def analyze_gene_expression_by_race(expression_df, merged_df, genes_of_interest):
    """
    Analyze gene expression differences by race.

    Parameters:
    - expression_df (DataFrame): Expression data.
    - merged_df (DataFrame): Merged phenotype and race data.
    - genes_of_interest (list): List of gene names to analyze.

    Returns:
    - None
    """
    race_column = "race"
    available_genes = expression_df.columns.tolist()
    genes_to_analyze = [gene for gene in genes_of_interest if gene in available_genes]
    missing_genes = set(genes_of_interest) - set(genes_to_analyze)

    if missing_genes:
        print(f"Skipping missing genes: {missing_genes}")
    if not genes_to_analyze:
        print("No genes to analyze. Exiting.")
        return

    for gene in genes_to_analyze:
        print(f"\nAnalyzing gene: {gene}")
        data = pd.merge(expression_df[['sample', gene]], merged_df[['sample', race_column]], on='sample', how='inner')
        data = data.dropna(subset=[gene, race_column])
        race_groups = data[race_column].unique()

        if len(race_groups) < 2:
            print("Not enough race groups for comparison. Skipping.")
            continue

        group_data = [data[data[race_column] == group][gene] for group in race_groups]
        if sum(len(g) >= 3 for g in group_data) < 2:
            print("Not enough data in groups for statistical comparison. Skipping.")
            continue

        f_stat, p_value = stats.f_oneway(*group_data)
        print(f"ANOVA results for {gene}: F-statistic = {f_stat:.4f}, P-value = {p_value:.4e}")

        plt.figure(figsize=(10, 6))
        sns.boxplot(x=race_column, y=gene, data=data)
        sns.swarmplot(x=race_column, y=gene, data=data, color=".25")
        plt.title(f"Expression of {gene} by Race\nANOVA P-value = {p_value:.4e}")
        plt.ylabel("Expression Level")
        plt.xlabel("Race")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [6]:
# Step 1: Define working directory and file paths
working_dir = "/data/expression/"
xml_dir = f"{working_dir}/xml_files"
phenotype_file = f"{working_dir}TcgaTargetGTEX_phenotype.txt"
expression_file = f"{working_dir}TcgaTargetGtex_RSEM_Hugo_norm_count"
genes_of_interest = ["MMP9"]

# Step 2: Load ethnicity data from XML files
race_df = load_race_data(xml_dir)

# Step 3: Load and merge phenotype data with ethnicity
phenotype_df = load_phenotype_data(phenotype_file, race_df)

# Step 4: Load expression data
samples = phenotype_df['sample'].tolist()  # Get the list of sample IDs from phenotype data
expression_df = load_expression_data(expression_file, samples, genes_of_interest)


# Optional: Print distinct ethnicities and their counts
print("Distinct races and their counts:")
print(phenotype_df['race'].value_counts())

# Step 5: Analyze gene expression by race


Reading phenotype data...
Merged phenotype data with race.
Filtering expression data...
Loading filtered data into DataFrame...
Distinct races and their counts:
Series([], Name: count, dtype: int64)


In [7]:
transposed_exp_df = expression_df.T
transposed_exp_df.columns = transposed_exp_df.iloc[0]
transposed_exp_df = transposed_exp_df[1:]
# reindex making the actual index named sample a column named sample
transposed_exp_df = transposed_exp_df.reset_index()
# rename the index to sample
transposed_exp_df = transposed_exp_df.rename(columns={'index': 'sample'})


In [8]:
analyze_gene_expression_by_race (transposed_exp_df, phenotype_df, genes_of_interest)


Analyzing gene: MMP9
Not enough race groups for comparison. Skipping.
